In [1]:
#PP1 PCB Detection and Merging

#Import the libraries
import pandas as pd
import numpy as np
import scipy as sp

import os
import sys
import time

import csv
import datetime

import importlib

from matplotlib import pyplot as plt
from matplotlib import dates  as md 
from matplotlib.colors import LogNorm

In [2]:
#Import routines
import routine_rawaccl_to_binryseqPP3  
import routine_get_pcb_durPP3          
import routine_elim_spur_pcbPP3        
import routine_get_pcb_level_dataPP3   
import routine_merging_algoPP3         
import routine_merge_pxmty_algoPP3

Start of definition of the rawaccl_to_binryseq procedure 2020-04-22 19:04:02.473259
Version currently used was compiled at 2019-06-25 17:08
Inputs are inp_vibr_df,downsample_rate,use_filt_accl_flag,use_quant_sig_flag,HYS_LOW_THRESH,HYS_HIGH_THRESH
End   of definition of the procedure 2020-04-22 19:04:02.473446
Start of defining procedure : 2020-04-22 19:04:02.474663
End   of defining procedure : 2020-04-22 19:04:02.474797
Start of defining procedure : 2020-04-22 19:04:02.476059
End   of defining procedure : 2020-04-22 19:04:02.476333
Start of defining procedure : 2020-04-22 19:04:02.477428
Version : 2019-04-23 12:50
End   of defining procedure : 2020-04-22 19:04:02.477593
Start of defining procedure : 2020-04-22 19:04:02.480266
Using version 2019-04-06 08:30
End   of defining procedure : 2020-04-22 19:04:02.480376
Start of defining procedure : 2020-04-22 19:04:02.481179
Version : 2019-04-09 21:59
End   of defining procedure : 2020-04-22 19:04:02.481245


In [3]:
pp3_vibr_file = pd.read_csv('PP3_IMU11.csv',usecols = ['time','ax','ay','az'])
pp3_vibr_file = pp3_vibr_file.sort_values(by="time")
pp3_vibr_file = pp3_vibr_file.reset_index()
pp3_vibr_file = pp3_vibr_file.assign(net_accl = np.sqrt(pp3_vibr_file['ax']**2 + pp3_vibr_file['ay']**2 + pp3_vibr_file['az']**2))

In [4]:
pp3_vibr_file['time']=pd.to_datetime(pp3_vibr_file['time'])
pp3_vibr_file=pp3_vibr_file.rename(columns={'time':'timestamp'})

In [5]:
pp3_vibr_file

,index,timestamp,ax,ay,az,net_accl
0,0,2020-03-11 02:00:00.038568960,-0.031067,1.039185,0.010437,1.039701
1,1,2020-03-11 02:00:00.039619072,-0.028809,1.040527,0.012390,1.041000
2,2,2020-03-11 02:00:00.040615936,-0.028809,1.040527,0.012390,1.041000
3,3,2020-03-11 02:00:00.041619968,-0.029114,1.039307,0.010742,1.039770
4,4,2020-03-11 02:00:00.042646016,-0.029114,1.039307,0.010742,1.039770
5,5,2020-03-11 02:00:00.075331072,-0.030823,1.040771,0.014099,1.041323
6,6,2020-03-11 02:00:00.076754944,-0.030029,1.041748,0.010132,1.042230
7,7,2020-03-11 02:00:00.137847040,-0.030029,1.041748,0.010132,1.042230
8,8,2020-03-11 02:00:00.138902784,-0.030029,1.041748,0.010132,1.042230
9,9,2020-03-11 02:00:00.139905024,-0.030029,1.041748,0.010132,1.042230


In [6]:
filt_sig_pp3 = routine_rawaccl_to_binryseqPP3.rawaccl_to_binryseq(pp3_vibr_file,1,'N')

START :  2020-04-22 19:04:17.220614
Mean      of normalized signal = 2.137252866039963e-11
Deviation of normalized signal = 0.9999999999999316
-------------------------------------------------------
Performing Wiener filtering


/home/raghu/anaconda3/lib/python3.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()


Performing downsampling by : 1
Performing quantization
Mode = 0.95
0 : 0.1 | 0
0.1 : 0.2 | 0
0.2 : 0.30000000000000004 | 0
0.30000000000000004 : 0.4 | 0
0.4 : 0.5 | 0
0.5 : 0.6 | 0
0.6 : 0.7 | 450
0.7 : 0.7999999999999999 | 110036
0.7999999999999999 : 0.8999999999999999 | 1168971
0.8999999999999999 : 0.9999999999999999 | 1376273
0.9999999999999999 : 1.0999999999999999 | 241309
1.0999999999999999 : 1.2 | 52689
1.2 : 1.3 | 49117
1.3 : 1.4000000000000001 | 117979
1.4000000000000001 : 1.5000000000000002 | 58156
1.5000000000000002 : 1.6000000000000003 | 45562
1.6000000000000003 : 1.7000000000000004 | 42208
1.7000000000000004 : 1.8000000000000005 | 41945
1.8000000000000005 : 1.9000000000000006 | 33334
1.9000000000000006 : 2.0000000000000004 | 24528
2.0000000000000004 : 2.1000000000000005 | 15293
2.1000000000000005 : 2.2000000000000006 | 8846
2.2000000000000006 : 2.3000000000000007 | 5514
2.3000000000000007 : 2.400000000000001 | 3409
2.400000000000001 : 2.500000000000001 | 3703
2.500000000000

In [7]:
pcb_data_pp3 = routine_get_pcb_durPP3.get_pcb_dur(filt_sig_pp3.binry_sig)

End of processing at i = 104 j = 104


In [8]:
filt_sig_pp3['cor_binry_sig'] = routine_elim_spur_pcbPP3.elim_spur_pcb(filt_sig_pp3.binry_sig,pcb_data_pp3,700)

/home/raghu/Desktop/TCSstreaming/iiotstream2/streaming2/PP3/routine_elim_spur_pcbPP3.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  cor_binry_sig[STR_IND:END_IND] = 0.0


In [9]:
pcb_data_pp3 = routine_get_pcb_durPP3.get_pcb_dur(filt_sig_pp3.cor_binry_sig)

End of processing at i = 83 j = 83


In [10]:
pcb_level_pp3 = routine_get_pcb_level_dataPP3.get_pcb_level_data(filt_sig_pp3,pcb_data_pp3,50)

START :  2020-04-22 19:05:50.875975
str_index = 447938
local_str = 0
end_index = 3384648
local_end = 82
Mode = 3425.0
local_mode = 3425.0
------------------------
Mode calculated at : 2020-04-22 19:05:51.111364
Number of boards detected        = 83
Machine utilization factor       = 6.735654967362701 %
Average   board active duration  = 39.57670682730924 secs
Median    board active duration  = 45.36 secs
Deviation board active duration  = 11.46582622181479 secs
END   :  2020-04-22 19:05:51.609099
Execution time = 0.012218733333333334 mins


In [11]:
pcb_level_pp3

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage
0,447938,2020-03-11 03:13:09.867908096,449247,2020-03-11 03:13:22.984514048,1,13.116606,0.0,0.382190
1,470043,2020-03-11 03:16:48.968409088,471195,2020-03-11 03:17:00.279441152,1,11.311032,0.0,0.336350
2,683212,2020-03-11 03:51:43.482625024,686908,2020-03-11 03:52:19.543802880,1,36.061178,0.0,1.079124
3,883361,2020-03-11 04:24:17.387753984,886751,2020-03-11 04:24:50.498308864,1,33.110555,0.0,0.989781
4,921834,2020-03-11 04:30:33.448342016,925150,2020-03-11 04:31:05.819348224,1,32.371006,0.0,0.968175
5,954223,2020-03-11 04:35:49.702797824,955625,2020-03-11 04:36:03.388624896,1,13.685827,0.0,0.409343
6,958709,2020-03-11 04:36:33.518579968,961124,2020-03-11 04:36:57.058631936,1,23.540052,0.0,0.705109
7,995418,2020-03-11 04:42:31.635399936,996809,2020-03-11 04:42:45.188581888,1,13.553182,0.0,0.406131
8,1002575,2020-03-11 04:43:41.410660096,1004437,2020-03-11 04:43:59.588965888,1,18.178306,0.0,0.543650
9,1012629,2020-03-11 04:45:19.497175040,1013708,2020-03-11 04:45:30.028184064,1,10.531009,0.0,0.315036


In [12]:
pcb_level_pp3.reset_index(inplace=True)
pcb_level_pp3

,index,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage
0,0,447938,2020-03-11 03:13:09.867908096,449247,2020-03-11 03:13:22.984514048,1,13.116606,0.0,0.382190
1,1,470043,2020-03-11 03:16:48.968409088,471195,2020-03-11 03:17:00.279441152,1,11.311032,0.0,0.336350
2,2,683212,2020-03-11 03:51:43.482625024,686908,2020-03-11 03:52:19.543802880,1,36.061178,0.0,1.079124
3,3,883361,2020-03-11 04:24:17.387753984,886751,2020-03-11 04:24:50.498308864,1,33.110555,0.0,0.989781
4,4,921834,2020-03-11 04:30:33.448342016,925150,2020-03-11 04:31:05.819348224,1,32.371006,0.0,0.968175
5,5,954223,2020-03-11 04:35:49.702797824,955625,2020-03-11 04:36:03.388624896,1,13.685827,0.0,0.409343
6,6,958709,2020-03-11 04:36:33.518579968,961124,2020-03-11 04:36:57.058631936,1,23.540052,0.0,0.705109
7,7,995418,2020-03-11 04:42:31.635399936,996809,2020-03-11 04:42:45.188581888,1,13.553182,0.0,0.406131
8,8,1002575,2020-03-11 04:43:41.410660096,1004437,2020-03-11 04:43:59.588965888,1,18.178306,0.0,0.543650
9,9,1012629,2020-03-11 04:45:19.497175040,1013708,2020-03-11 04:45:30.028184064,1,10.531009,0.0,0.315036


In [13]:
pcb_level_pp3.drop('index', axis=1, inplace=True)

In [14]:
tempdf=pcb_level_pp3
tempdf['nmodes']=0
tempdf

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage,nmodes
0,447938,2020-03-11 03:13:09.867908096,449247,2020-03-11 03:13:22.984514048,1,13.116606,0.0,0.382190,0
1,470043,2020-03-11 03:16:48.968409088,471195,2020-03-11 03:17:00.279441152,1,11.311032,0.0,0.336350,0
2,683212,2020-03-11 03:51:43.482625024,686908,2020-03-11 03:52:19.543802880,1,36.061178,0.0,1.079124,0
3,883361,2020-03-11 04:24:17.387753984,886751,2020-03-11 04:24:50.498308864,1,33.110555,0.0,0.989781,0
4,921834,2020-03-11 04:30:33.448342016,925150,2020-03-11 04:31:05.819348224,1,32.371006,0.0,0.968175,0
5,954223,2020-03-11 04:35:49.702797824,955625,2020-03-11 04:36:03.388624896,1,13.685827,0.0,0.409343,0
6,958709,2020-03-11 04:36:33.518579968,961124,2020-03-11 04:36:57.058631936,1,23.540052,0.0,0.705109,0
7,995418,2020-03-11 04:42:31.635399936,996809,2020-03-11 04:42:45.188581888,1,13.553182,0.0,0.406131,0
8,1002575,2020-03-11 04:43:41.410660096,1004437,2020-03-11 04:43:59.588965888,1,18.178306,0.0,0.543650,0
9,1012629,2020-03-11 04:45:19.497175040,1013708,2020-03-11 04:45:30.028184064,1,10.531009,0.0,0.315036,0


In [15]:
mod_actv_dur = routine_get_pcb_level_dataPP3.get_mode(pcb_data_pp3.pcb_actv_dur,50)/100

Mode = 3425.0


In [16]:
mod_actv_dur 

34.25

In [17]:
tseries = np.zeros(tempdf.shape[0])
tseries

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [18]:
for i, row in pcb_level_pp3.iterrows():
    nmodes=row.proc_dur/mod_actv_dur
    frac=nmodes%1
    nmodes=np.floor(nmodes)
    
    if (1-frac) <= .15:
        nmodes=np.floor(nmodes)+1
    
    if nmodes==1.0:
        continue
    tseries[i] = nmodes
tempdf['nmodes'] = tseries

In [19]:
tseries

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [20]:
temp_df3=tempdf[tempdf['nmodes']>1]

In [21]:
temp_df3

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage,nmodes


In [22]:
split_times=[]
for i, row in temp_df3.iterrows():
    nmodes=int(row.nmodes)
    split_times.append(np.linspace(row.arvl_tmstmp.to_datetime64().astype('int'),row.dptr_tmstmp.to_datetime64().astype('int'),num=row.nmodes+1))

In [23]:
split_times

[]

In [24]:
temp_df3.reset_index(inplace=True)

In [25]:
temp_df3

,index,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage,nmodes


In [26]:
PP_boardeventsSplit = pd.DataFrame()
for i, row in temp_df3.iterrows():
    nmodes=int(row.nmodes)
    wt=row.proc_dur/nmodes
#     en=row.energy/nmodes
#     if nmodes%2==0:
    for j in range(len(split_times[i])-1):
        ts1=pd.to_datetime(split_times[i][j])
        ts2=pd.to_datetime(split_times[i][j+1])
        PP_boardeventsSplit=PP_boardeventsSplit.append({"arvl_index":row.arvl_index, "arvl_tmstmp":ts1, "dptr_index":row.dptr_index, "dptr_tmstmp":ts2, "no_of_parts":1, "proc_dur": wt, "maint_dur":row.maint_dur, "weightage":wt/mod_actv_dur, "nmodes":1}, ignore_index=True)

In [27]:
PP_boardeventsSplit

""


In [28]:
pcb_level_pp3.drop(tempdf[tempdf['nmodes']>1].index, inplace=True)

In [29]:
pcb_level_pp3

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage,nmodes
0,447938,2020-03-11 03:13:09.867908096,449247,2020-03-11 03:13:22.984514048,1,13.116606,0.0,0.382190,0.0
1,470043,2020-03-11 03:16:48.968409088,471195,2020-03-11 03:17:00.279441152,1,11.311032,0.0,0.336350,0.0
2,683212,2020-03-11 03:51:43.482625024,686908,2020-03-11 03:52:19.543802880,1,36.061178,0.0,1.079124,0.0
3,883361,2020-03-11 04:24:17.387753984,886751,2020-03-11 04:24:50.498308864,1,33.110555,0.0,0.989781,0.0
4,921834,2020-03-11 04:30:33.448342016,925150,2020-03-11 04:31:05.819348224,1,32.371006,0.0,0.968175,0.0
5,954223,2020-03-11 04:35:49.702797824,955625,2020-03-11 04:36:03.388624896,1,13.685827,0.0,0.409343,0.0
6,958709,2020-03-11 04:36:33.518579968,961124,2020-03-11 04:36:57.058631936,1,23.540052,0.0,0.705109,0.0
7,995418,2020-03-11 04:42:31.635399936,996809,2020-03-11 04:42:45.188581888,1,13.553182,0.0,0.406131,0.0
8,1002575,2020-03-11 04:43:41.410660096,1004437,2020-03-11 04:43:59.588965888,1,18.178306,0.0,0.543650,0.0
9,1012629,2020-03-11 04:45:19.497175040,1013708,2020-03-11 04:45:30.028184064,1,10.531009,0.0,0.315036,0.0


In [30]:
pcb_level_pp3=pcb_level_pp3.append(PP_boardeventsSplit, ignore_index=True)

In [31]:
pcb_level_pp3

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage,nmodes
0,447938,2020-03-11 03:13:09.867908096,449247,2020-03-11 03:13:22.984514048,1,13.116606,0.0,0.382190,0.0
1,470043,2020-03-11 03:16:48.968409088,471195,2020-03-11 03:17:00.279441152,1,11.311032,0.0,0.336350,0.0
2,683212,2020-03-11 03:51:43.482625024,686908,2020-03-11 03:52:19.543802880,1,36.061178,0.0,1.079124,0.0
3,883361,2020-03-11 04:24:17.387753984,886751,2020-03-11 04:24:50.498308864,1,33.110555,0.0,0.989781,0.0
4,921834,2020-03-11 04:30:33.448342016,925150,2020-03-11 04:31:05.819348224,1,32.371006,0.0,0.968175,0.0
5,954223,2020-03-11 04:35:49.702797824,955625,2020-03-11 04:36:03.388624896,1,13.685827,0.0,0.409343,0.0
6,958709,2020-03-11 04:36:33.518579968,961124,2020-03-11 04:36:57.058631936,1,23.540052,0.0,0.705109,0.0
7,995418,2020-03-11 04:42:31.635399936,996809,2020-03-11 04:42:45.188581888,1,13.553182,0.0,0.406131,0.0
8,1002575,2020-03-11 04:43:41.410660096,1004437,2020-03-11 04:43:59.588965888,1,18.178306,0.0,0.543650,0.0
9,1012629,2020-03-11 04:45:19.497175040,1013708,2020-03-11 04:45:30.028184064,1,10.531009,0.0,0.315036,0.0


In [32]:
pcb_level_pp3 = pcb_level_pp3.sort_values(by="arvl_tmstmp")

In [33]:
pcb_level_pp3.reset_index(inplace=True)

In [34]:
pcb_level_pp3

,index,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage,nmodes
0,0,447938,2020-03-11 03:13:09.867908096,449247,2020-03-11 03:13:22.984514048,1,13.116606,0.0,0.382190,0.0
1,1,470043,2020-03-11 03:16:48.968409088,471195,2020-03-11 03:17:00.279441152,1,11.311032,0.0,0.336350,0.0
2,2,683212,2020-03-11 03:51:43.482625024,686908,2020-03-11 03:52:19.543802880,1,36.061178,0.0,1.079124,0.0
3,3,883361,2020-03-11 04:24:17.387753984,886751,2020-03-11 04:24:50.498308864,1,33.110555,0.0,0.989781,0.0
4,4,921834,2020-03-11 04:30:33.448342016,925150,2020-03-11 04:31:05.819348224,1,32.371006,0.0,0.968175,0.0
5,5,954223,2020-03-11 04:35:49.702797824,955625,2020-03-11 04:36:03.388624896,1,13.685827,0.0,0.409343,0.0
6,6,958709,2020-03-11 04:36:33.518579968,961124,2020-03-11 04:36:57.058631936,1,23.540052,0.0,0.705109,0.0
7,7,995418,2020-03-11 04:42:31.635399936,996809,2020-03-11 04:42:45.188581888,1,13.553182,0.0,0.406131,0.0
8,8,1002575,2020-03-11 04:43:41.410660096,1004437,2020-03-11 04:43:59.588965888,1,18.178306,0.0,0.543650,0.0
9,9,1012629,2020-03-11 04:45:19.497175040,1013708,2020-03-11 04:45:30.028184064,1,10.531009,0.0,0.315036,0.0


In [35]:
pcb_level_pp3.drop('index', axis=1, inplace=True)
pcb_level_pp3.drop('nmodes', axis=1, inplace=True)

In [36]:
pcb_level_pp3

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage
0,447938,2020-03-11 03:13:09.867908096,449247,2020-03-11 03:13:22.984514048,1,13.116606,0.0,0.382190
1,470043,2020-03-11 03:16:48.968409088,471195,2020-03-11 03:17:00.279441152,1,11.311032,0.0,0.336350
2,683212,2020-03-11 03:51:43.482625024,686908,2020-03-11 03:52:19.543802880,1,36.061178,0.0,1.079124
3,883361,2020-03-11 04:24:17.387753984,886751,2020-03-11 04:24:50.498308864,1,33.110555,0.0,0.989781
4,921834,2020-03-11 04:30:33.448342016,925150,2020-03-11 04:31:05.819348224,1,32.371006,0.0,0.968175
5,954223,2020-03-11 04:35:49.702797824,955625,2020-03-11 04:36:03.388624896,1,13.685827,0.0,0.409343
6,958709,2020-03-11 04:36:33.518579968,961124,2020-03-11 04:36:57.058631936,1,23.540052,0.0,0.705109
7,995418,2020-03-11 04:42:31.635399936,996809,2020-03-11 04:42:45.188581888,1,13.553182,0.0,0.406131
8,1002575,2020-03-11 04:43:41.410660096,1004437,2020-03-11 04:43:59.588965888,1,18.178306,0.0,0.543650
9,1012629,2020-03-11 04:45:19.497175040,1013708,2020-03-11 04:45:30.028184064,1,10.531009,0.0,0.315036


In [37]:
pcb_merge_pp3 = pcb_level_pp3
mrg_binry_pp3 = filt_sig_pp3.cor_binry_sig

THRESHOLD = 0.95
HIGH_END  = 1.50

iter_one = 0
iter_two = 0

In [38]:
pcb_merge_pp3

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage
0,447938,2020-03-11 03:13:09.867908096,449247,2020-03-11 03:13:22.984514048,1,13.116606,0.0,0.382190
1,470043,2020-03-11 03:16:48.968409088,471195,2020-03-11 03:17:00.279441152,1,11.311032,0.0,0.336350
2,683212,2020-03-11 03:51:43.482625024,686908,2020-03-11 03:52:19.543802880,1,36.061178,0.0,1.079124
3,883361,2020-03-11 04:24:17.387753984,886751,2020-03-11 04:24:50.498308864,1,33.110555,0.0,0.989781
4,921834,2020-03-11 04:30:33.448342016,925150,2020-03-11 04:31:05.819348224,1,32.371006,0.0,0.968175
5,954223,2020-03-11 04:35:49.702797824,955625,2020-03-11 04:36:03.388624896,1,13.685827,0.0,0.409343
6,958709,2020-03-11 04:36:33.518579968,961124,2020-03-11 04:36:57.058631936,1,23.540052,0.0,0.705109
7,995418,2020-03-11 04:42:31.635399936,996809,2020-03-11 04:42:45.188581888,1,13.553182,0.0,0.406131
8,1002575,2020-03-11 04:43:41.410660096,1004437,2020-03-11 04:43:59.588965888,1,18.178306,0.0,0.543650
9,1012629,2020-03-11 04:45:19.497175040,1013708,2020-03-11 04:45:30.028184064,1,10.531009,0.0,0.315036


In [39]:
pp3_pxstr_file = pd.read_csv('PP3_prox11.csv',usecols=['time','PP3Entry'])
pp3_pxstr_file=pp3_pxstr_file.rename(columns={'time':'timestamp'})
pp3_pxstr_file

,timestamp,PP3Entry
0,2020-03-11T03:51:40.410372096Z,1
1,2020-03-11T03:51:40.607119104Z,1
2,2020-03-11T03:51:40.778969088Z,1
3,2020-03-11T04:24:14.110148864Z,1
4,2020-03-11T04:30:29.137856Z,1
5,2020-03-11T04:30:29.327860224Z,1
6,2020-03-11T04:35:45.957735936Z,1
7,2020-03-11T04:35:46.157113088Z,1
8,2020-03-11T04:42:27.528956928Z,1
9,2020-03-11T04:42:27.70444288Z,1


In [40]:
PROC_ONE_END_FLAG = 0
while (PROC_ONE_END_FLAG == 0):
    
    iter_one = iter_one + 1
    print('PROC-ONE-ITERATION :', iter_one)
    
    pcb_merge_pp3,mrg_binry_pp3,no_of_merged_pcbs = routine_merge_pxmty_algoPP3.merge_pxmty_algo    \
    (inp_df        = pcb_merge_pp3                                      \
    ,pxmty_df      = pp3_pxstr_file      \
    ,inp_binry_sig = mrg_binry_pp3                         \
    ,THRESHOLD     = THRESHOLD                                               \
    ,HIGH_END      = HIGH_END                                                \
    )
    
    if (no_of_merged_pcbs == 0):
        PROC_ONE_END_FLAG = 1
    #end-if
    
    print ('=============================================')

PROC-ONE-ITERATION : 1
START :  2020-04-22 19:05:55.343619
0 | 2020-03-11T03:12:54.867908096 : 2020-03-11T03:13:19.867908096 | 0
1 | 2020-03-11T03:16:33.968409088 : 2020-03-11T03:16:58.968409088 | 0
5 | 2020-03-11T04:35:34.702797824 : 2020-03-11T04:35:59.702797824 | 2
6 | 2020-03-11T04:36:23.518579968 : 2020-03-11T04:36:43.518579968 | 0
i = 5 | merge_flag = 1
7 | 2020-03-11T04:42:16.635399936 : 2020-03-11T04:42:41.635399936 | 2
8 | 2020-03-11T04:43:31.410660096 : 2020-03-11T04:43:51.410660096 | 0
i = 7 | merge_flag = 1
9 | 2020-03-11T04:45:04.497175040 : 2020-03-11T04:45:29.497175040 | 0
39 | 2020-03-11T05:33:35.367753984 : 2020-03-11T05:34:00.367753984 | 2
40 | 2020-03-11T05:34:01.616611072 : 2020-03-11T05:34:21.616611072 | 0
i = 39 | merge_flag = 1
54 | 2020-03-11T06:02:58.307928064 : 2020-03-11T06:03:23.307928064 | 2
55 | 2020-03-11T06:07:10.902810112 : 2020-03-11T06:07:30.902810112 | 2
55 | 2020-03-11T06:07:05.902810112 : 2020-03-11T06:07:30.902810112 | 2
62 | 2020-03-11T06:14:33.6

In [41]:
PROC_TWO_END_FLAG = 0
while(PROC_TWO_END_FLAG == 0):
    
    iter_two = iter_two + 1
    print('PROC-TWO-ITERATION :', iter_two)
    
    pcb_merge_pp3,mrg_binry_pp3,no_of_merged_pcbs = routine_merging_algoPP3.merging_algo  \
    (inp_df        = pcb_merge_pp3                                   \
    ,inp_binry_sig = mrg_binry_pp3                                   \
    ,THRESHOLD     = THRESHOLD                                            \
    ,HIGH_END      = HIGH_END                                             \
    ) 
    
    if(no_of_merged_pcbs == 0):
        PROC_TWO_END_FLAG = 1
    #end-if
    
    print ('=============================================')

PROC-TWO-ITERATION : 1
START :  2020-04-22 19:05:56.286893
i = 0 | merge_flag = 1
i = 50 | merge_flag = 1
i = 58 | merge_flag = 1
i = 62 | merge_flag = 1
i = 74 | merge_flag = 1
No. of merged PCBS = 5
END   :  2020-04-22 19:05:56.604706
Execution time = 0.005296883333333334 mins
PROC-TWO-ITERATION : 2
START :  2020-04-22 19:05:56.604972
No. of merged PCBS = 0
END   :  2020-04-22 19:05:56.874036
Execution time = 0.0044844 mins


In [42]:
pcb_merge_pp3

,arvl_index,arvl_tmstmp,dptr_index,dptr_tmstmp,no_of_parts,proc_dur,maint_dur,weightage
0,683212,2020-03-11 03:51:43.482625024,686908,2020-03-11 03:52:19.543802880,1,36.061178,0.000000,1.079124
1,883361,2020-03-11 04:24:17.387753984,886751,2020-03-11 04:24:50.498308864,1,33.110555,0.000000,0.989781
2,921834,2020-03-11 04:30:33.448342016,925150,2020-03-11 04:31:05.819348224,1,32.371006,0.000000,0.968175
3,954223,2020-03-11 04:35:49.702797824,961124,2020-03-11 04:36:57.058631936,2,37.225879,30.129955,1.114453
4,995418,2020-03-11 04:42:31.635399936,1013708,2020-03-11 04:45:30.028184064,3,42.262497,136.130287,1.264818
5,1059785,2020-03-11 04:52:59.749822976,1063209,2020-03-11 04:53:33.183724032,1,33.433901,0.000000,0.999708
6,1067350,2020-03-11 04:54:13.600209920,1070684,2020-03-11 04:54:46.163683840,1,32.563474,0.000000,0.973431
7,1072283,2020-03-11 04:55:01.722873088,1075662,2020-03-11 04:55:34.765824000,1,33.042951,0.000000,0.986569
8,1092708,2020-03-11 04:58:21.548579072,1096261,2020-03-11 04:58:56.205990912,1,34.657412,0.000000,1.037372
9,1101497,2020-03-11 04:59:47.251666944,1104915,2020-03-11 05:00:20.643721984,1,33.392055,0.000000,0.997956


In [43]:
filt_sig_pp3['mrg_binry_sig'] = mrg_binry_pp3

In [44]:
pp3EMfile = pd.read_csv('PP3_EM11.csv', usecols=['time', 'c1', 'c2', 'c3'])

In [45]:
raw_pp_current=pp3EMfile

In [46]:
raw_pp_current['time']=pd.to_datetime(raw_pp_current['time'])
raw_pp_current=raw_pp_current.rename(columns={'time':'timestamp'})

In [47]:
raw_pp_current

,timestamp,c1,c2,c3
0,2020-03-11 02:00:01.360014848,0.0,0.0,0.0
1,2020-03-11 02:00:03.215964928,0.0,0.0,0.0
2,2020-03-11 02:00:05.055578112,0.0,0.0,0.0
3,2020-03-11 02:00:06.879211776,0.0,0.0,0.0
4,2020-03-11 02:00:08.703341056,0.0,0.0,0.0
5,2020-03-11 02:00:10.558552064,0.0,0.0,0.0
6,2020-03-11 02:00:12.398160896,0.0,0.0,0.0
7,2020-03-11 02:00:14.254002176,0.0,0.0,0.0
8,2020-03-11 02:00:16.109498880,0.0,0.0,0.0
9,2020-03-11 02:00:17.965431808,0.0,0.0,0.0


In [48]:
raw_pp_current['total_current']=raw_pp_current['c1']+raw_pp_current['c2']+raw_pp_current['c3']

In [49]:
working_times_df=pd.DataFrame()

working_times_df['timestamp']=pcb_merge_pp3['arvl_tmstmp']
working_times_df['working_time']=pcb_merge_pp3['proc_dur']
working_times_df['event']=1

In [50]:
working_times_df

,timestamp,working_time,event
0,2020-03-11 03:51:43.482625024,36.061178,1
1,2020-03-11 04:24:17.387753984,33.110555,1
2,2020-03-11 04:30:33.448342016,32.371006,1
3,2020-03-11 04:35:49.702797824,37.225879,1
4,2020-03-11 04:42:31.635399936,42.262497,1
5,2020-03-11 04:52:59.749822976,33.433901,1
6,2020-03-11 04:54:13.600209920,32.563474,1
7,2020-03-11 04:55:01.722873088,33.042951,1
8,2020-03-11 04:58:21.548579072,34.657412,1
9,2020-03-11 04:59:47.251666944,33.392055,1


In [51]:
working_times_df['energy'] = pcb_merge_pp3.apply(lambda x: raw_pp_current.loc[(raw_pp_current.timestamp <= x.dptr_tmstmp) & (x.arvl_tmstmp <= raw_pp_current.timestamp), ['total_current']].sum()*230/3600000, axis=1)

In [52]:
working_times_df

,timestamp,working_time,event,energy
0,2020-03-11 03:51:43.482625024,36.061178,1,0.003962
1,2020-03-11 04:24:17.387753984,33.110555,1,0.003432
2,2020-03-11 04:30:33.448342016,32.371006,1,0.003959
3,2020-03-11 04:35:49.702797824,37.225879,1,0.006099
4,2020-03-11 04:42:31.635399936,42.262497,1,0.012347
5,2020-03-11 04:52:59.749822976,33.433901,1,0.004213
6,2020-03-11 04:54:13.600209920,32.563474,1,0.004035
7,2020-03-11 04:55:01.722873088,33.042951,1,0.003117
8,2020-03-11 04:58:21.548579072,34.657412,1,0.003508
9,2020-03-11 04:59:47.251666944,33.392055,1,0.002882


In [53]:
PP_temp_idle=filt_sig_pp3[['timestamp', 'mrg_binry_sig']]
PP_temp_idle['mrg_binry_sig']=PP_temp_idle['mrg_binry_sig']*-1

/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [54]:
PP_temp_idle

,timestamp,mrg_binry_sig
0,2020-03-11 02:00:01.991347200,-0.0
1,2020-03-11 02:00:01.992357888,-0.0
2,2020-03-11 02:00:02.026961152,-0.0
3,2020-03-11 02:00:02.038039808,-0.0
4,2020-03-11 02:00:02.039087104,-0.0
5,2020-03-11 02:00:02.040097024,-0.0
6,2020-03-11 02:00:02.041102848,-0.0
7,2020-03-11 02:00:02.075737856,-0.0
8,2020-03-11 02:00:02.077516032,-0.0
9,2020-03-11 02:00:02.078573824,-0.0


In [55]:
PP_temp_idle['mrg_binry_sig'].iloc[0]=(PP_temp_idle['mrg_binry_sig'].iloc[1]+1)%2*-1
PP_temp_idle['mrg_binry_sig'].iloc[-1]=(PP_temp_idle['mrg_binry_sig'].iloc[-2]+1)%2*-1

/home/raghu/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [56]:
PP_temp_idle

,timestamp,mrg_binry_sig
0,2020-03-11 02:00:01.991347200,-1.0
1,2020-03-11 02:00:01.992357888,-0.0
2,2020-03-11 02:00:02.026961152,-0.0
3,2020-03-11 02:00:02.038039808,-0.0
4,2020-03-11 02:00:02.039087104,-0.0
5,2020-03-11 02:00:02.040097024,-0.0
6,2020-03-11 02:00:02.041102848,-0.0
7,2020-03-11 02:00:02.075737856,-0.0
8,2020-03-11 02:00:02.077516032,-0.0
9,2020-03-11 02:00:02.078573824,-0.0


In [57]:
idle_times_raw=sp.signal.find_peaks(PP_temp_idle.mrg_binry_sig, height=(-0.4), width=1)
idle_times_raw_df=pd.DataFrame({"sample_number":idle_times_raw[0], "working_time":idle_times_raw[1]['widths']/100})

In [58]:
idle_times_raw_df

,sample_number,working_time
0,223969,4479.37
1,577203,2120.17
2,785134,1964.53
3,904292,350.83
4,939686,290.73
5,978270,342.94
6,1036746,460.77
7,1065279,41.41
8,1071483,15.99
9,1084184,170.46


In [59]:
for x, row in idle_times_raw_df.iterrows():
    idle_times_raw_df.ix[x,'start_time']=PP_temp_idle.iloc[int(idle_times_raw[1]['left_ips'][x])].timestamp
    idle_times_raw_df.ix[x,'end_time']=PP_temp_idle.iloc[int(idle_times_raw[1]['right_ips'][x])].timestamp
    idle_times_raw_df.ix[x,'timestamp']=PP_temp_idle.iloc[int(idle_times_raw[0][x])].timestamp

/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
/home/raghu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after remo

In [60]:
idle_times_raw_df['event']=0

In [61]:
idle_times_raw_df['energy'] = idle_times_raw_df.apply(lambda x: raw_pp_current.loc[(raw_pp_current.timestamp <= x.end_time) & 
                                                            (x.start_time <= raw_pp_current.timestamp),
                                                            ['total_current']].sum()*230/3600000, axis=1)

In [62]:
idle_times_raw_df

,sample_number,working_time,start_time,end_time,timestamp,event,energy
0,223969,4479.37,2020-03-11 02:00:01.991347200,2020-03-11 03:13:09.823883008,2020-03-11 02:36:33.317783040,0,0.028130
1,577203,2120.17,2020-03-11 03:17:00.235323904,2020-03-11 03:51:43.481633024,2020-03-11 03:34:27.738629120,0,0.133774
2,785134,1964.53,2020-03-11 03:52:19.499810048,2020-03-11 04:24:17.386061824,2020-03-11 04:08:18.558573056,0,0.110879
3,904292,350.83,2020-03-11 04:24:50.487691008,2020-03-11 04:30:33.399853056,2020-03-11 04:27:41.900788992,0,0.021946
4,939686,290.73,2020-03-11 04:31:05.813822976,2020-03-11 04:35:49.701796096,2020-03-11 04:33:27.724806144,0,0.018245
5,978270,342.94,2020-03-11 04:36:57.014205952,2020-03-11 04:42:31.634351872,2020-03-11 04:39:44.329651200,0,0.021541
6,1036746,460.77,2020-03-11 04:45:30.013995776,2020-03-11 04:52:59.748843008,2020-03-11 04:49:15.050968832,0,0.028388
7,1065279,41.41,2020-03-11 04:53:33.182116096,2020-03-11 04:54:13.599212032,2020-03-11 04:53:53.379430912,0,0.002597
8,1071483,15.99,2020-03-11 04:54:46.132029184,2020-03-11 04:55:01.721876992,2020-03-11 04:54:53.947151872,0,0.001081
9,1084184,170.46,2020-03-11 04:55:34.731375872,2020-03-11 04:58:21.502452992,2020-03-11 04:56:57.992605952,0,0.010781


In [63]:
working_times_df=working_times_df[['event', 'working_time', 'energy','timestamp']]
idle_times_df=idle_times_raw_df[['event', 'working_time', 'energy', 'timestamp']]

In [64]:
working_times_df

,event,working_time,energy,timestamp
0,1,36.061178,0.003962,2020-03-11 03:51:43.482625024
1,1,33.110555,0.003432,2020-03-11 04:24:17.387753984
2,1,32.371006,0.003959,2020-03-11 04:30:33.448342016
3,1,37.225879,0.006099,2020-03-11 04:35:49.702797824
4,1,42.262497,0.012347,2020-03-11 04:42:31.635399936
5,1,33.433901,0.004213,2020-03-11 04:52:59.749822976
6,1,32.563474,0.004035,2020-03-11 04:54:13.600209920
7,1,33.042951,0.003117,2020-03-11 04:55:01.722873088
8,1,34.657412,0.003508,2020-03-11 04:58:21.548579072
9,1,33.392055,0.002882,2020-03-11 04:59:47.251666944


In [65]:
idle_times_df

,event,working_time,energy,timestamp
0,0,4479.37,0.028130,2020-03-11 02:36:33.317783040
1,0,2120.17,0.133774,2020-03-11 03:34:27.738629120
2,0,1964.53,0.110879,2020-03-11 04:08:18.558573056
3,0,350.83,0.021946,2020-03-11 04:27:41.900788992
4,0,290.73,0.018245,2020-03-11 04:33:27.724806144
5,0,342.94,0.021541,2020-03-11 04:39:44.329651200
6,0,460.77,0.028388,2020-03-11 04:49:15.050968832
7,0,41.41,0.002597,2020-03-11 04:53:53.379430912
8,0,15.99,0.001081,2020-03-11 04:54:53.947151872
9,0,170.46,0.010781,2020-03-11 04:56:57.992605952


In [66]:
idle_times_raw_df['energy'].sum(axis = 0, skipna = True) 

1.39157475

In [67]:
working_times_df['energy'].sum(axis = 0, skipna = True)

0.3072601944444444

In [68]:
from influxdb import InfluxDBClient
import json

IFhost="localhost"
IFport=8086
IFdbname = 'IIOT'

client = InfluxDBClient(host=IFhost, port=IFport,database=IFdbname)

In [69]:
events_batch=[]

# events_batch = json.loads(LL_events.to_json())

for i,row in idle_times_df.iterrows():
    events_batch.append({ 'measurement': 'EVENTS','tags':{'Machine':'PP3'}, 'time':row.timestamp ,'fields': {'PP3 Idle Event':int(row.event),'PP3 Idle Time':float(row.working_time),"PP3 Idle Energy":float(row.energy)}})
    
for i,row in working_times_df.iterrows():
    events_batch.append({ 'measurement': 'EVENTS','tags':{'Machine':'PP3'}, 'time':row.timestamp ,'fields': {'PP3 Processing Event':int(row.event),'PP3 Processing Time':float(row.working_time),"PP3 Processing Energy":float(row.energy)}})

In [70]:
events_batch

[{'measurement': 'EVENTS',
  'tags': {'Machine': 'PP3'},
  'time': Timestamp('2020-03-11 02:36:33.317783040'),
  'fields': {'PP3 Idle Event': 0,
   'PP3 Idle Time': 4479.37,
   'PP3 Idle Energy': 0.028130277777777775}},
 {'measurement': 'EVENTS',
  'tags': {'Machine': 'PP3'},
  'time': Timestamp('2020-03-11 03:34:27.738629120'),
  'fields': {'PP3 Idle Event': 0,
   'PP3 Idle Time': 2120.17,
   'PP3 Idle Energy': 0.1337743888888889}},
 {'measurement': 'EVENTS',
  'tags': {'Machine': 'PP3'},
  'time': Timestamp('2020-03-11 04:08:18.558573056'),
  'fields': {'PP3 Idle Event': 0,
   'PP3 Idle Time': 1964.53,
   'PP3 Idle Energy': 0.11087852777777778}},
 {'measurement': 'EVENTS',
  'tags': {'Machine': 'PP3'},
  'time': Timestamp('2020-03-11 04:27:41.900788992'),
  'fields': {'PP3 Idle Event': 0,
   'PP3 Idle Time': 350.83,
   'PP3 Idle Energy': 0.02194647222222222}},
 {'measurement': 'EVENTS',
  'tags': {'Machine': 'PP3'},
  'time': Timestamp('2020-03-11 04:33:27.724806144'),
  'fields': {'

In [71]:
client.write_points(events_batch,batch_size=len(idle_times_df))
client.write_points(events_batch,batch_size=len(working_times_df))

True

In [72]:
PP_events=pd.DataFrame()

In [73]:
PP_events=PP_events.append(working_times_df)

In [74]:
PP_events=PP_events.append(idle_times_df)

In [75]:
PP_events

,event,working_time,energy,timestamp
0,1,36.061178,0.003962,2020-03-11 03:51:43.482625024
1,1,33.110555,0.003432,2020-03-11 04:24:17.387753984
2,1,32.371006,0.003959,2020-03-11 04:30:33.448342016
3,1,37.225879,0.006099,2020-03-11 04:35:49.702797824
4,1,42.262497,0.012347,2020-03-11 04:42:31.635399936
5,1,33.433901,0.004213,2020-03-11 04:52:59.749822976
6,1,32.563474,0.004035,2020-03-11 04:54:13.600209920
7,1,33.042951,0.003117,2020-03-11 04:55:01.722873088
8,1,34.657412,0.003508,2020-03-11 04:58:21.548579072
9,1,33.392055,0.002882,2020-03-11 04:59:47.251666944
